# Step 1: Getting data from API and converting in Appropriate structure

In [16]:
#import api data
import requests
url= 'https://api.covidactnow.org/v2/counties.json?apiKey=6abf0fcadd5d4ddab5f43d619f68871e'
resp = requests.get(url=url).json()

In [17]:
#check responce
resp[61]

{'fips': '01065',
 'country': 'US',
 'state': 'AL',
 'county': 'Hale County',
 'hsa': '156',
 'hsaName': 'Tuscaloosa (Tuscaloosa), AL - Pickens, AL',
 'level': 'county',
 'lat': None,
 'locationId': 'iso1:us#iso2:us-al#fips:01065',
 'long': None,
 'population': 14651,
 'hsaPopulation': 252047,
 'metrics': {'testPositivityRatio': 0.017,
  'testPositivityRatioDetails': {'source': 'other'},
  'caseDensity': 9.3,
  'weeklyNewCasesPer100k': 64.8,
  'contactTracerCapacityRatio': None,
  'infectionRate': 1.11,
  'infectionRateCI90': 1.1,
  'icuCapacityRatio': None,
  'bedsWithCovidPatientsRatio': 0.0,
  'weeklyCovidAdmissionsPer100k': 0.8,
  'vaccinationsInitiatedRatio': 0.727,
  'vaccinationsCompletedRatio': 0.613,
  'vaccinationsAdditionalDoseRatio': 0.249},
 'riskLevels': {'overall': 2,
  'testPositivityRatio': 0,
  'caseDensity': 1,
  'contactTracerCapacityRatio': 4,
  'infectionRate': 2,
  'icuCapacityRatio': 4},
 'cdcTransmissionLevel': 2,
 'communityLevels': {'cdcCommunityLevel': 0, 'c

In [20]:
#get fips and other relevant attributes from the api
for i in resp:
     val = i['riskLevels']['overall']
     fips_c = i['fips']
     inf = i['metrics']['infectionRate']
     county = i['county']
     state = i['state']
     coun = i['county']
     vac = i['metrics']['vaccinationsInitiatedRatio']
     i.clear()
     i['fips'] = fips_c
     i['risk'] = val
     i['infectionRate'] = inf
     i['county'] = county
     i['vaccinationsInitiatedRatio'] = vac

In [21]:
resp[1]

{'fips': '02016',
 'risk': 1,
 'infectionRate': 1.0,
 'county': 'Aleutians West Census Area',
 'vaccinationsInitiatedRatio': 0.746}

In [22]:
#output data without hospitals file
import json
with open('datawithoutHos.json', 'w') as f:
    json.dump(resp, f)

# Step 2: Refine hospital data from different source

In [ ]:
#import health data
dataH = json.load(open('dataMapHealth.json'))
dataH = dataH['features']

In [23]:
#refine data
resultArray = []
for i in dataH:
    my_dict = i['properties']
    del my_dict['State']
    del my_dict['County Name']
    del my_dict['ICU Bed Source']
    del my_dict['ICU Bed Source Last Updated']
    my_dict['fips'] = my_dict.pop('fips_code')
    data = i['properties']
    resultArray.append(data)

In [ ]:
#hospital json file
import json
with open('hospital2.json', 'w') as f:
    json.dump(resultArray, f)

# Step 3 Join datas from both sources with fips code as reference

In [24]:

#import health data
dataR = json.load(open('datawithoutHos.json'))
#import health data
dataH = json.load(open('hospital2.json'))

In [25]:
import pandas as pd
dr = pd.DataFrame(dataR)
dh = pd.DataFrame(dataH)

In [27]:
dataH

[{'Staffed All Beds': 55.0,
  'Staffed ICU Beds': 6.0,
  'Licensed All Beds': 85.0,
  'All Bed Occupancy Rate': 0.62,
  'ICU Bed Occupancy Rate': 0.53,
  'Population': 55601.0,
  'Population (20+)': 41177.0,
  'Population (65+)': 8653.0,
  'Staffed All Beds [Per 1000 People]': 0.989,
  'Staffed All Beds [Per 1000 Adults (20+)]': 1.336,
  'Staffed All Beds [Per 1000 Elderly (65+)]': 6.356,
  'Staffed ICU Beds [Per 1000 People]': 0.108,
  'Staffed ICU Beds [Per 1000 Adults (20+)]': 0.146,
  'Staffed ICU Beds [Per 1000 Elderly (65+)]': 0.693,
  'Licensed All Beds [Per 1000 People]': 1.529,
  'Licensed All Beds [Per 1000 Adults (20+)]': 2.064,
  'Licensed All Beds [Per 1000 Elderly (65+)]': 9.823,
  'fips': '01001'},
 {'Staffed All Beds': 25.0,
  'Staffed ICU Beds': 6.0,
  'Licensed All Beds': 25.0,
  'All Bed Occupancy Rate': 0.5,
  'ICU Bed Occupancy Rate': 0.34,
  'Population': 57840.0,
  'Population (20+)': 43175.0,
  'Population (65+)': 10548.0,
  'Staffed All Beds [Per 1000 People]':

In [28]:
#combine both the hospital and risk data in one json file 
count = 0
for i in dataR:
    fips = i['fips']
    for j in dataH:
        if j['fips'] == fips:
            i["Staffed All Beds [Per 1000 Adults (20+)]"] = j["Staffed All Beds [Per 1000 Adults (20+)]"]
            i["Staffed ICU Beds [Per 1000 Adults (20+)]"] = j["Staffed ICU Beds [Per 1000 Adults (20+)]"]
            i["Licensed All Beds [Per 1000 Adults (20+)]"] = j["Licensed All Beds [Per 1000 Adults (20+)]"]

In [30]:
dataR

[{'fips': '02013',
  'risk': 2,
  'infectionRate': 1.03,
  'county': 'Aleutians East Borough',
  'vaccinationsInitiatedRatio': 0.851},
 {'fips': '02016',
  'risk': 1,
  'infectionRate': 1.0,
  'county': 'Aleutians West Census Area',
  'vaccinationsInitiatedRatio': 0.746,
  'Staffed All Beds [Per 1000 Adults (20+)]': 0.0,
  'Staffed ICU Beds [Per 1000 Adults (20+)]': 0.0,
  'Licensed All Beds [Per 1000 Adults (20+)]': 0.0},
 {'fips': '02020',
  'risk': 3,
  'infectionRate': 0.98,
  'county': 'Anchorage Municipality',
  'vaccinationsInitiatedRatio': 0.757,
  'Staffed All Beds [Per 1000 Adults (20+)]': 4.585,
  'Staffed ICU Beds [Per 1000 Adults (20+)]': 0.342,
  'Licensed All Beds [Per 1000 Adults (20+)]': 5.382},
 {'fips': '02050',
  'risk': 3,
  'infectionRate': 0.71,
  'county': 'Bethel Census Area',
  'vaccinationsInitiatedRatio': 0.723,
  'Staffed All Beds [Per 1000 Adults (20+)]': 3.068,
  'Staffed ICU Beds [Per 1000 Adults (20+)]': 0.0,
  'Licensed All Beds [Per 1000 Adults (20+)]

In [31]:
#final json file
import json
with open('hospitalandrisk.json', 'w') as f:
    json.dump(dataR, f)